<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



# **Resolución de consignas planteadas para el desafío 2**

Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado

Recuerden que su notebook de entrega debe poder correrse de inicio a fin sin la aparición de errores.





*   Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.
*   Elegir términos de interés y buscar términos más similares y menos similares.
Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones.


*   Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.
*   Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

  








In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 45.3 MB/s eta 0:00:00


### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-23 21:16:07--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-23 21:16:07--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-23 21:16:07--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['r-kelly.txt',
 'nickelback.txt',
 'joni-mitchell.txt',
 'dolly-parton.txt',
 'britney-spears.txt',
 'eminem.txt',
 'radiohead.txt',
 'missy-elliott.txt',
 'jimi-hendrix.txt',
 'paul-simon.txt',
 'amy-winehouse.txt',
 'johnny-cash.txt',
 'bob-marley.txt',
 'notorious_big.txt',
 'bob-dylan.txt',
 'drake.txt',
 'janisjoplin.txt',
 'lorde.txt',
 'cake.txt',
 'Kanye_West.txt',
 'lil-wayne.txt',
 'nursery_rhymes.txt',
 'al-green.txt',
 'dj-khaled.txt',
 'nirvana.txt',
 'kanye.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'nicki-minaj.txt',
 'adele.txt',
 'disney.txt',
 'kanye-west.txt',
 'prince.txt',
 'bruce-springsteen.txt',
 'dr-seuss.txt',
 'Lil_Wayne.txt',
 'notorious-big.txt',
 'blink-182.txt',
 'michael-jackson.txt',
 'alicia-keys.txt',
 'ludacris.txt',
 'lady-gaga.txt',
 'beatles.txt',
 'bjork.txt',
 'dickinson.txt',
 'leonard-cohen.txt',
 'patti-smith.txt',
 'lin-manuel-miranda.txt',
 'bieber.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/lady-gaga.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-4107314280.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/lady-gaga.txt', sep='/n', header=None)


,0
0,"I'll undress you, 'cause you're tired"
1,Cover you as you desire
2,When you fall asleep inside my arms
3,May not have the fancy things
4,But I'll give you everything


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3807


### 1 - Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:5]

[["i'll", 'undress', 'you', "'cause", "you're", 'tired'],
 ['cover', 'you', 'as', 'you', 'desire'],
 ['when', 'you', 'fall', 'asleep', 'inside', 'my', 'arms'],
 ['may', 'not', 'have', 'the', 'fancy', 'things'],
 ['but', "i'll", 'give', 'you', 'everything']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
w2v_model

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 3807


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 722


In [ ]:
w2v_model.wv.index_to_key

['i',
 'you',
 'the',
 'a',
 'me',
 'my',
 'to',
 'and',
 'oh',
 "i'm",
 'your',
 'that',
 'love',
 'be',
 'in',
 'on',
 'it',
 "don't",
 'want',
 'just',
 'eh',
 'with',
 'baby',
 'of',
 'but',
 'we',
 'what',
 'wanna',
 'do',
 'like',
 'know',
 'so',
 'is',
 'this',
 "i'll",
 'can',
 "you're",
 "can't",
 'not',
 'could',
 'p',
 'up',
 'one',
 'good',
 'no',
 'got',
 'as',
 'm',
 'for',
 'if',
 'million',
 'way',
 "it's",
 'reasons',
 'all',
 'when',
 'heart',
 'dance',
 'hair',
 'gonna',
 'out',
 "'cause",
 'la',
 'girl',
 'ah',
 'show',
 'face',
 'boys',
 'muh',
 'need',
 'yeah',
 'touch',
 'he',
 'body',
 'was',
 'ha',
 'come',
 'make',
 'down',
 'give',
 'tell',
 "i've",
 'take',
 'were',
 'read',
 'she',
 'poker',
 "won't",
 'see',
 'na',
 'night',
 'doo',
 'say',
 'man',
 'how',
 'boy',
 'bad',
 'am',
 'at',
 'call',
 'something',
 'get',
 'time',
 'feel',
 'teeth',
 'now',
 'tonight',
 'da',
 'let',
 'have',
 "let's",
 'name',
 'are',
 'put',
 'or',
 'ooh',
 'gaga',
 'go',
 'ga

### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 95949.125
Loss after epoch 1: 94438.484375
Loss after epoch 2: 92134.921875
Loss after epoch 3: 89348.6875
Loss after epoch 4: 87581.78125
Loss after epoch 5: 86386.5
Loss after epoch 6: 82604.5
Loss after epoch 7: 83551.9375
Loss after epoch 8: 82420.1875
Loss after epoch 9: 80938.25
Loss after epoch 10: 80617.375
Loss after epoch 11: 80232.6875
Loss after epoch 12: 77376.0625
Loss after epoch 13: 77069.75
Loss after epoch 14: 75925.875
Loss after epoch 15: 76315.75
Loss after epoch 16: 75938.625
Loss after epoch 17: 76163.75
Loss after epoch 18: 75317.625
Loss after epoch 19: 75326.75


(372127, 606000)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["bitch"], topn=10)

[('rich', 0.7835507392883301),
 ('move', 0.6836098432540894),
 ('work', 0.6773135662078857),
 ('crazy', 0.6414573788642883),
 ("who's", 0.6208381056785583),
 ('bub', 0.6196054816246033),
 ('ecstasy', 0.6144267320632935),
 ('chick', 0.6131560802459717),
 ('slam', 0.6087528467178345),
 ("sippin'", 0.6083977222442627)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["judas"], topn=10)

[('wait', -0.04984201863408089),
 ('give', -0.0958671048283577),
 ('go', -0.09691119939088821),
 ('to', -0.10609272867441177),
 ('let', -0.11521956324577332),
 ('yo', -0.12100647389888763),
 ('these', -0.1275346875190735),
 ("won't", -0.1370234340429306),
 ('here', -0.13752731680870056),
 ('anymore', -0.13811874389648438)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[("wasn't", 0.5403189063072205),
 ("doin'", 0.5284400582313538),
 ('mistaken', 0.5248053669929504),
 ('game', 0.5203059315681458),
 ('fame', 0.5199912786483765),
 ('wonder', 0.5006205439567566),
 ('goddess', 0.499702513217926),
 ('judas', 0.49916738271713257),
 ('juda', 0.49748459458351135),
 ('whoa', 0.4940335154533386)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('fancy', 0.8555483818054199),
 ('aha', 0.8468345999717712),
 ('eh', 0.8340397477149963),
 ("that's", 0.8336365222930908),
 ('faster', 0.8172706961631775)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["dum"])

[('around', -0.10955024510622025),
 ('left', -0.12162521481513977),
 ("'cause", -0.13543613255023956),
 ('the', -0.13988982141017914),
 ('love', -0.1533007025718689),
 ('we', -0.15713950991630554),
 ('never', -0.16237430274486542),
 ('thing', -0.16659915447235107),
 ('in', -0.166707381606102),
 ('to', -0.16814610362052917)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 1.10445321e-01  6.44204259e-01  7.37232789e-02  3.33778530e-01
  1.45718247e-01  2.41831437e-01  3.12752247e-01  2.18885690e-02
 -2.28874683e-01 -2.52560340e-02  2.39715755e-01 -1.40500098e-01
  2.48901054e-01  3.07230353e-02 -1.86449364e-01  5.99745810e-02
  1.16390139e-01  4.45784032e-01  8.30405653e-02 -1.18065707e-01
 -2.45500594e-01 -1.15323022e-01  2.48597294e-01  4.21240687e-01
 -2.14707240e-01  1.17134616e-01 -2.33313739e-01 -3.98074612e-02
 -8.25403258e-02 -2.73012482e-02  9.80285630e-02 -1.06961802e-01
  9.78340358e-02  1.98472053e-01 -2.54989415e-01  3.81633282e-01
  3.20491076e-01 -3.80331650e-02 -3.06205332e-01  2.39003792e-01
 -2.05868244e-01 -1.43413067e-01  2.60927051e-01  3.28057855e-01
  3.14355999e-01  1.14090212e-01  2.30749454e-02 -5.81368357e-02
  7.66068250e-02  4.57477003e-01 -2.38316149e-01 -4.37814780e-02
  1.80292249e-01  1.19229816e-01  4.19040956e-02 -2.95587569e-01
 -4.23516363e-01  1.69636935e-01 -1.98163409e-02 -1.42256320e-01
 -4.26552743e-02 -2.58778

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ("wasn't", 0.5403188467025757),
 ("doin'", 0.5284400582313538),
 ('mistaken', 0.5248053669929504),
 ('game', 0.520305871963501),
 ('fame', 0.5199912786483765),
 ('wonder', 0.5006205439567566),
 ('goddess', 0.49970248341560364),
 ('judas', 0.49916738271713257),
 ('juda', 0.49748459458351135)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[("wasn't", 0.5403189063072205),
 ("doin'", 0.5284400582313538),
 ('mistaken', 0.5248053669929504),
 ('game', 0.5203059315681458),
 ('fame', 0.5199912786483765),
 ('wonder', 0.5006205439567566),
 ('goddess', 0.499702513217926),
 ('judas', 0.49916738271713257),
 ('juda', 0.49748459458351135),
 ('whoa', 0.4940335154533386)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

#**Conclusiones generales del desafío (para el cierre del notebook)**

En este desafío entrené un modelo Word2Vec a partir de un corpus compuesto por letras de canciones de Lady Gaga, generando representaciones vectoriales capaces de capturar relaciones semánticas entre palabras. A partir del modelo entrenado se realizaron consultas de similitud, reducción de dimensionalidad mediante t-SNE y visualización de los embeddings, lo que permitió explorar la estructura interna del vocabulario derivado del corpus.

Los resultados obtenidos muestran que, a pesar de trabajar con un corpus relativamente acotado, el modelo logra capturar patrones temáticos reales presentes en la discografía del artista. Las consultas de similitud revelan asociaciones coherentes entre palabras. El uso de t-SNE para proyectar los embeddings en dos dimensiones permitió identificar grupos de palabras que reflejan distintos ejes semánticos que emergen de las letras: amor, emociones, fama, actitud, narrativa religiosa y expresiones asociadas a identidad o empoderamiento.

La visualización pone en evidencia que las palabras no se distribuyen de manera aleatoria, sino que tienden a formar zonas densas que responden a temas recurrentes en el repertorio. Esto sugiere que el modelo logró aprender relaciones contextuales significativamente bien considerando el tamaño del corpus.

Va el resumen de clusters encontrado:



* Cluster 1: born was way play game
* Cluster 2: call alejandro name don´t
* Cluster 3: fix body want with there's away
* Cluster 4: monster ate heart